In [1]:
import pyodbc
import pandas as pd
import getpass

In [2]:
# Specify database connection parameters
server = "ifw9ecos-bvdb11.fws.doi.net\ecos_beta"
database = "IRIS_DataWarehouse"

# Uncomment if not using Windows authentication
#username = input()
#password = getpass.getpass()

Leana adding this as a test

In [3]:
##----------------------------------------------------
# Create a database connection object using a Data Source Name (DSN)
# defined by the Windows ODBC Data Source Administrator
##----------------------------------------------------
#dsn_name = ""
#db_connection = pyodbc.connect("DSN="+dsn_name+";UID="+username+";PWD="+password)

##----------------------------------------------------
#  Create a database connection object by specifying connection parameters
##----------------------------------------------------

# Uncomment if not using Windows authentication
#db_connection = pyodbc.connect("\
#    DRIVER={SQL Server};\
#    SERVER=" + server + ";\
#    PORT=1443;\
#    DATABASE=" + database + ";\
#    UID=" + username + ";\
#    PWD=" + password)

db_connection = pyodbc.connect("\
    DRIVER={SQL Server};\
    SERVER=" + server + ";\
    PORT=1443;\
    DATABASE=" + database + ";\
    Trusted_Connection=yes")

InterfaceError: ('28000', "[28000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed for user 'IFW\\lgoetze'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC SQL Server Driver]Invalid connection string attribute (0); [28000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed for user 'IFW\\lgoetze'. (18456); [28000] [Microsoft][ODBC SQL Server Driver]Invalid connection string attribute (0)")

In [ ]:
cursor = db_connection.cursor()

In [ ]:
#PRIMRsurveyID = 'FF01RCOP00-003'
#PRIMRsurveyID = 'FF01RPLM00-007'
PRIMRsurveyID = 'FF01RORG00-072'
PRINT(PRIMRsurveyID)

In [ ]:
queryIdentification = "SELECT [SurveyCode],[Station],[SurveyName], [SurveyType],[ResourceThemeLevel1],[ResourceThemeLevel2] ,[AnswerQ1],[AnswerQ2],[AnswerQ3],[AnswerQ4],[SurveyCoordinatorName],[SurveyCoordinatorTitle],[StationName],[SurveyArea],[ManagementUnits] FROM [IRIS_DataWarehouse].[Refuges].[VW_Surveys] WHERE [SurveyCode] = '" + PRIMRsurveyID + "';"
dfIdentification = pd.read_sql(queryIdentification, db_connection)
dfIdentification

In [7]:
queryKeywords = "SELECT Refuges.DimKeyword.Name as theme FROM Refuges.DimKeyword RIGHT OUTER JOIN Refuges.DimSurvey INNER JOIN Refuges.FactSurveys ON Refuges.DimSurvey.ID = Refuges.FactSurveys.SurveyID INNER JOIN Refuges.FactSurveyKeywords ON Refuges.FactSurveys.SurveyID = Refuges.FactSurveyKeywords.SurveyID ON Refuges.DimKeyword.ID = Refuges.FactSurveyKeywords.KeywordID WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
dfKeywords = pd.read_sql(queryKeywords, db_connection)
dfKeywords
#dfKeywords.to_csv("keywords.csv")

,theme
0,Pacific Seabird Program


In [8]:
querySurveyCoordinator = "SELECT Refuges.DimSurvey.SurveyCode, Refuges.DimSurvey.SurveyCoordinatorName, DimStaff.Title, DimStaff.OrganizationCode, DimStaff.City, DimStaff.StateCode, DimStaff.ZipCode, DimStaff.PhoneNumber, DimStaff.CellNumber, DimStaff.Email, DimStaff.Office, DimStaff.Department, DimOrganization.OrgCode FROM FactStaff INNER JOIN DimStaff ON FactStaff.StaffID = DimStaff.ID AND FactStaff.StaffID = DimStaff.ID AND FactStaff.StaffID = DimStaff.ID INNER JOIN DimOrganization ON FactStaff.OrganizationID = DimOrganization.ID AND FactStaff.OrganizationID = DimOrganization.ID AND FactStaff.OrganizationID = DimOrganization.ID RIGHT OUTER JOIN Refuges.DimSurvey ON DimStaff.FullName = Refuges.DimSurvey.SurveyCoordinatorName WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
dfSurveyCoordinator = pd.read_sql(querySurveyCoordinator, db_connection)
dfSurveyCoordinator
#dfSurveyCoordinator.to_csv("surveycoordinator.csv")


,SurveyCode,SurveyCoordinatorName,Title,OrganizationCode,City,StateCode,ZipCode,PhoneNumber,CellNumber,Email,Office,Department,OrgCode
0,FF01RORG00-072,Shawn Stephensen,Wildlife Biologist,FF01ROCC00,Newport,OR,97365,541 867-4550,541-270-1863,Shawn_Stephensen@fws.gov,"Newport, OR",Refuges,13594


In [9]:
queryBioticGroup = "SELECT Refuges.DimSurvey.SurveyCode, Refuges.DimBioticGroup.ITIS_TSN, Refuges.DimBioticGroup.ITIS_TSN_Parent, Refuges.DimBioticGroup.ScientificName_Parent, Refuges.DimBioticGroup.CommonName_Parent,     Refuges.DimBioticGroup.Rank_Parent FROM Refuges.FactSurveyBioticGroups INNER JOIN Refuges.DimSurvey ON Refuges.FactSurveyBioticGroups.SurveyID = Refuges.DimSurvey.ID INNER JOIN Refuges.DimBioticGroup ON Refuges.FactSurveyBioticGroups.BioticGroupID = Refuges.DimBioticGroup.ID WHERE Refuges.DimSurvey.SurveyCode = '" + PRIMRsurveyID + "';"
dfBioticGroup = pd.read_sql(queryBioticGroup, db_connection)
dfBioticGroup
#dfBioticGroup.to_csv("bioticgroup.csv")

,SurveyCode,ITIS_TSN,ITIS_TSN_Parent,ScientificName_Parent,CommonName_Parent,Rank_Parent


In [10]:
from pymdwizard.core.xml_utils import XMLRecord, XMLNode #import in this manner requires adding a path file to your site-packages directory

test = "C:/Users/kso/Documents/Git/PRIMR_to_FGDC/PRIMR_Metadata_Template_20220429.xml"
original_md = XMLRecord(test)

In [11]:
# one-to-one is easy
# edit individual node text
original_md.metadata.idinfo.citation.citeinfo.title.text = dfIdentification.at[0,'SurveyName']
original_md.metadata.idinfo.ptcontac.cntinfo.cntorgp.cntper.text = dfSurveyCoordinator.at[0,'SurveyCoordinatorName']
original_md.metadata.idinfo.ptcontac.cntinfo.cntpos.text = dfSurveyCoordinator.at[0,'Title']

In [12]:
original_md

<metadata>
  <idinfo>
    <citation>
      <citeinfo>
        <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
        <origin>Other Contact Name [not available in IRIS warehouse]</origin>
        <pubdate>
        </pubdate>
        <title>Tufted Puffin - Breeding Distribution and Abundance</title>
        <edition>PRIMRproduct_v1 [what is this?]</edition>
        <geoform>
        </geoform>
        <pubinfo>
          <pubplace>CityName, StateName from DimCity</pubplace>
          <publish>U.S. Fish and Wildlife Service</publish>
        </pubinfo>
        <onlink>URL to ServCat Survey Products [not available in IRIS warehouse]</onlink>
        <lworkcit>
          <citeinfo>
            <origin>SurveyCoordinatorName from Refuges.DimSurvey</origin>
            <pubdate>
            </pubdate>
            <title>SurveyName from Refuges.DimSurvey</title>
            <edition>PRIMRproduct_v1 [what is this?]</edition>
            <geoform>
            </geoform>
           

In [13]:
dfKeywordsSet = (
    pd.DataFrame(dfIdentification)
    .loc[:, ['ResourceThemeLevel1', 'ResourceThemeLevel2', 'SurveyType', 'SurveyCode']]
    .melt()
    .drop(columns=['variable'])
    .rename(columns={'value':'theme'})
    .append(dfKeywords)
    .append((
        pd.DataFrame(dfBioticGroup)
        .loc[:, ['ScientificName_Parent']]
        .melt()
        .drop(columns=['variable'])
        .rename(columns={'value':'theme'})),
        ignore_index=True
    )
)
dfKeywordsSet

,theme
0,Biological Integrity
1,Other Biota
2,Monitoring to Inform Management
3,FF01RORG00-072
4,Pacific Seabird Program


In [14]:
kw = original_md.metadata.idinfo.keywords
kw.clear_children('theme')
kw

<keywords>
  <place>
    <placekt>FWS-PRIMR</placekt>
    <placekey>Survey  Area</placekey>
    <placekey>Management Units</placekey>
  </place>
</keywords>

In [15]:
def createXMLNode_from_df(df, subelement_tag, optional_element_tag):
    xml_data = ['<root>']
    for column in df.columns:
        xml_data.append('<{}>'.format(column))  # Opening element tag
        xml_data.append(optional_element_tag.format(column))
        for field in df.index:
            # writing sub-elements
            xml_data.append('<temp_tag>{1}</temp_tag>'.format(field, df[column][field]))
        xml_data.append('</{}>'.format(column))  # Closing element tag
    xml_data.append('</root>')

    my_string = ""
    for i in xml_data:
        my_string += ('{}').format(i)

    new_string = (
        my_string
            .replace('<root>','')
            .replace('</root>','')
            .replace('temp_tag',subelement_tag)
        )

    return(XMLNode(new_string))

In [16]:
newKW = createXMLNode_from_df(dfKeywordsSet, "themekey", "<themekt>FWS-PRIMR</themekt>")
newKW

<theme>
  <themekt>FWS-PRIMR</themekt>
  <themekey>Biological Integrity</themekey>
  <themekey>Other Biota</themekey>
  <themekey>Monitoring to Inform Management</themekey>
  <themekey>FF01RORG00-072</themekey>
  <themekey>Pacific Seabird Program</themekey>
</theme>

In [17]:
kw.add_child(newKW)
kw

<keywords>
  <place>
    <placekt>FWS-PRIMR</placekt>
    <placekey>Survey  Area</placekey>
    <placekey>Management Units</placekey>
  </place>
  <theme>
    <themekt>FWS-PRIMR</themekt>
    <themekey>Biological Integrity</themekey>
    <themekey>Other Biota</themekey>
    <themekey>Monitoring to Inform Management</themekey>
    <themekey>FF01RORG00-072</themekey>
    <themekey>Pacific Seabird Program</themekey>
  </theme>
</keywords>

In [18]:
# write to new xml
original_md.save(fname='PRIMR_Metadata_Template_20220429Testing.xml')